In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score)

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\23324\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\23324\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Importing the datasets
# Import the training dataset with only the important colums
train_df = pd.read_csv("datasets/train.csv")

# Column 3 is the column of interest in the testing set
test_df = pd.read_csv("datasets/test.csv")

In [4]:
#Explore train dataset
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [6]:
# Cleaning the dataset

def clean_text(text):
    if isinstance(text, str):
        # Remove non-alphanumeric characters and convert to lowercase
        text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

        # Tokenize the text into individual words
        words = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

        # Join the words back into a single string
        clean_text = ' '.join(words)

        return clean_text


In [7]:
# Applying preprocessing function on the text column of the training dataset
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

# Converting sentiments to numerical form
sentiment_mapping = {'neutral': 0, 'positive': 1, 'negative': -1}
train_df['sentiment'] = train_df['sentiment'].map(sentiment_mapping)
print("Number of rows before dropna:", train_df.shape[0])

# Dropping rows with missing values
train_df.dropna(inplace=True)
print("Number of rows after dropna:", train_df.shape[0])

# Print the preprocessed train dataset
print(train_df.head())

Number of rows before dropna: 27481
Number of rows after dropna: 27480
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text  sentiment  \
0  I`d have responded, if I were going          0   
1                             Sooo SAD         -1   
2                          bullying me         -1   
3                       leave me alone         -1   
4                        Sons of ****,         -1   

                       cleaned_text  
0                   responded going  
1           sooo sad miss san diego  
2                     boss bullying  
3             interview leave alone  
4  sons put

In [8]:
# Splitting the training dataset into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(train_df.cleaned_text, train_df.sentiment, test_size=0.2, random_state=24)


In [9]:
# Initialising the TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

vectorizer.fit(list(X_train) + list(X_test))
X_train_vectorizer =  vectorizer.transform(X_train) 
X_test_vectorizer = vectorizer.transform(X_test)


In [10]:
log_reg = LogisticRegression(random_state=0, multi_class='multinomial', max_iter=1000)
log_reg.fit(X_train_vectorizer, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=0)

In [11]:
val_pred = log_reg.predict(X_test_vectorizer)
print("The accurary of logistic regression is",accuracy_score(list(y_test), val_pred))

The accurary of logistic regression is 0.6857714701601164


In [12]:
# Finding out which TF-IDF features contribute the most to determining the sentiment label
feature_names = vectorizer.get_feature_names_out()
# Coefficients indicate the importance of each feature in the classification
feature_importance = log_reg.coef_
feature_importance_df = pd.DataFrame(feature_importance, columns=feature_names)
print(feature_importance_df)

          0        00       000        01        03        04        05  \
0 -0.043028  0.539402 -0.216714  0.001622  0.244835  0.225411  0.087108   
1  0.170857 -0.098659 -0.052156  0.200661  0.018805 -0.165755 -0.038555   
2 -0.127829 -0.440744  0.268870 -0.202283 -0.263640 -0.059656 -0.048553   

         06        07        08  ...        ze   zealand      zero       zoe  \
0  0.313249 -0.056644 -0.243518  ...  0.140226  0.014948 -0.494884 -0.187081   
1 -0.021825  0.209610  0.205157  ... -0.219648 -0.147208  0.461392  0.244079   
2 -0.291424 -0.152965  0.038361  ...  0.079422  0.132260  0.033492 -0.056998   

     zombie   zombies      zone       zoo      zulu      zzzz  
0  0.095797  0.031117  0.013116 -0.326490 -0.311350  0.008246  
1 -0.031871 -0.048884  0.281774  0.247602 -0.005105 -0.096620  
2 -0.063926  0.017767 -0.294890  0.078888  0.316455  0.088374  

[3 rows x 12796 columns]


In [13]:
num_top_features = 500

# Create a dictionary to store the top features for each sentiment label
top_features_per_sentiment = {
    -1: feature_importance_df.loc[0].nlargest(num_top_features).index.tolist(),
    0: feature_importance_df.loc[1].nlargest(num_top_features).index.tolist(),
    1: feature_importance_df.loc[2].nlargest(num_top_features).index.tolist()
}

for index, row in train_df.iterrows():
    text = row['text']
    sentiment_label = int(row['sentiment'])  # Convert sentiment label to integer
    
    if sentiment_label in top_features_per_sentiment:
        top_features = top_features_per_sentiment[sentiment_label]  # All features for the sentiment label
        sentiment_parts = [feature for feature in top_features if feature in text]
    else:
        sentiment_parts = []  # No top features available for the sentiment label
    
    print(f"Sentiment parts for row {index}: {sentiment_parts}")

Sentiment parts for row 0: ['ed', 'goin', 'going', 'go', 'sp', 'de']
Sentiment parts for row 1: ['miss']
Sentiment parts for row 2: []
Sentiment parts for row 3: ['nt', 'ew']
Sentiment parts for row 4: ['ugh', 'already', 'c']
Sentiment parts for row 5: ['less', 'http', 'fo', 'un', 'th']
Sentiment parts for row 6: ['fun', 'smile', 'g', 'smiles', 'mi']
Sentiment parts for row 7: ['oooo']
Sentiment parts for row 8: ['th']
Sentiment parts for row 9: ['cool', 'hehe']
Sentiment parts for row 10: ['ko', 'chan', 'go', 'th']
Sentiment parts for row 11: ['g', 'like']
Sentiment parts for row 12: []
Sentiment parts for row 13: ['lost', 'nt', 'c']
Sentiment parts for row 14: ['th']
Sentiment parts for row 15: ['burned', 'sunburned', 'oh', 'sunburn', 'burn']
Sentiment parts for row 16: ['sigh', 'iv', 'na']
Sentiment parts for row 17: ['sick', 'nt', 'ew', 'yu', 'c']
Sentiment parts for row 18: ['miss', 'every', 'c', 'na']
Sentiment parts for row 19: ['th']
Sentiment parts for row 20: []
Sentiment par

Sentiment parts for row 649: ['ed', 'sp']
Sentiment parts for row 650: ['th']
Sentiment parts for row 651: []
Sentiment parts for row 652: ['nt']
Sentiment parts for row 653: ['fun', 'funny', 'haha', 'lol', 'ahaha', 'aha']
Sentiment parts for row 654: ['ouch', 'c']
Sentiment parts for row 655: ['ugh', 'worse', 'even', 'thought', 'life', 'c']
Sentiment parts for row 656: ['th']
Sentiment parts for row 657: ['g', 'lol', 'would', 'grad']
Sentiment parts for row 658: ['10 minutes', 'fo', 'th']
Sentiment parts for row 659: ['cute', 'g', 'ever', 'night']
Sentiment parts for row 660: ['happy', 'g', 'day', 'ever']
Sentiment parts for row 661: ['ah']
Sentiment parts for row 662: ['home', 'th']
Sentiment parts for row 663: ['man', 'inbox', 'c']
Sentiment parts for row 664: ['ppl', 'c']
Sentiment parts for row 665: ['sorry', 'sick', 'nt', 'c']
Sentiment parts for row 666: ['nice']
Sentiment parts for row 667: ['ed', 'renewed', 'th']
Sentiment parts for row 668: ['love', 'mothers', 'win', 'g', 'da

Sentiment parts for row 1563: ['g', 'tool']
Sentiment parts for row 1564: ['due', 'im', 'nt', 'iv', 'nerd', 'mean', 'c']
Sentiment parts for row 1565: ['th', 'de']
Sentiment parts for row 1566: ['die', 'dragged']
Sentiment parts for row 1567: ['ugh', 'enough', 'oh', 'good enough']
Sentiment parts for row 1568: ['g']
Sentiment parts for row 1569: ['drugs', 'workin', 'th', 'rug']
Sentiment parts for row 1570: ['th', 'de']
Sentiment parts for row 1571: ['nt', 'c']
Sentiment parts for row 1572: ['nt', 'jo', 'mental', 'c']
Sentiment parts for row 1573: ['life']
Sentiment parts for row 1574: ['dm', 'din', 'dinner', 'fo', 'th']
Sentiment parts for row 1575: ['sweet']
Sentiment parts for row 1576: ['sa', '45', 'hey', 'eye', 'yes', 'go', 'th']
Sentiment parts for row 1577: ['sa', 'private', 'cart', 'carter', 'youtube', 'th', 'pr', 'de']
Sentiment parts for row 1578: ['good', 'wow', 'g']
Sentiment parts for row 1579: ['gorgeous', 'g', 'day', 'rs']
Sentiment parts for row 1580: ['g', 'day', 'owe'

Sentiment parts for row 2294: []
Sentiment parts for row 2295: ['g']
Sentiment parts for row 2296: ['home', 'go', 'fo', 'un', 'th', 'pr']
Sentiment parts for row 2297: ['ed', 'till', 'fo', 'th']
Sentiment parts for row 2298: ['headache', 'ache', 'head', 'c']
Sentiment parts for row 2299: []
Sentiment parts for row 2300: []
Sentiment parts for row 2301: ['th']
Sentiment parts for row 2302: ['cry', 'crying', 'pain', 'painful', 'wat', 'c']
Sentiment parts for row 2303: ['fo', 'coffee', 'de']
Sentiment parts for row 2304: ['sucks', 'suck', 'nt', 'c']
Sentiment parts for row 2305: ['play', 'go', 'un', 'th', 'de']
Sentiment parts for row 2306: ['pr']
Sentiment parts for row 2307: ['empty', 'ed', 'cleaning', 'fo', 'th', 'de']
Sentiment parts for row 2308: ['sucks', 'suck', 'c']
Sentiment parts for row 2309: ['dead', 'rly']
Sentiment parts for row 2310: ['beautiful', 'g', 'day', 'rs']
Sentiment parts for row 2311: ['tell', 'th']
Sentiment parts for row 2312: ['upset', 'c']
Sentiment parts for 

Sentiment parts for row 3033: ['welcome', 'g', 'ever']
Sentiment parts for row 3034: ['sad']
Sentiment parts for row 3035: []
Sentiment parts for row 3036: ['wish', 'joy', 'g', 'day', 'ever', 'owe', 'however']
Sentiment parts for row 3037: ['sorry', 'im', 'oh', 'c']
Sentiment parts for row 3038: ['good', 'fun', 'funny', 'haha', 'g', 'aha']
Sentiment parts for row 3039: ['fo']
Sentiment parts for row 3040: ['hell', 'ate', 'c', 'na']
Sentiment parts for row 3041: ['th']
Sentiment parts for row 3042: ['awesome', 'watched']
Sentiment parts for row 3043: ['cant', 'nt', 'get home', 'c']
Sentiment parts for row 3044: ['love', 'welcome']
Sentiment parts for row 3045: []
Sentiment parts for row 3046: ['wake', 'breakfast', 'ah', 'th']
Sentiment parts for row 3047: ['thanks', 'thank', 'interesting', '3', 'g', 'day', 'rest', '33']
Sentiment parts for row 3048: ['ah', 'go', 'fo']
Sentiment parts for row 3049: ['tho', 'insurance', 'know', 'fo', 'un', 'th']
Sentiment parts for row 3050: ['g', 'mom', 

...

Sentiment parts for row 23269: ['g', 'curious', 'yes', 'treat', 'like']
Sentiment parts for row 23270: ['airport', 'ed', 'closed', 'hey', 'th']
Sentiment parts for row 23271: ['headache', 'ugh', 'ache', 'ughh', 'head', 'ughhh', 'c']
Sentiment parts for row 23272: ['losing', 'c']
Sentiment parts for row 23273: ['hello', 'goin', 'going', 'go']
Sentiment parts for row 23274: ['sa']
Sentiment parts for row 23275: ['excited', 'best', 'day']
Sentiment parts for row 23276: ['beautiful', 'wait', 'g', 'lol', 'day', 'guys', 'rs', 'aha']
Sentiment parts for row 23277: ['till', 'morning']
Sentiment parts for row 23278: ['tho', 'ed', 'less', 'go', 'fo', 'th']
Sentiment parts for row 23279: ['stupid', 'boo', 'nt', 'wont', 'facebook', 'c']
Sentiment parts for row 23280: ['g', 'grad', 'aha']
Sentiment parts for row 23281: ['till', 'stay', 'fo', 'th']
Sentiment parts for row 23282: []
Sentiment parts for row 23283: ['din', 'th']
Sentiment parts for row 23284: ['3', 'g', 'gooood']
Sentiment parts for ro

Sentiment parts for row 24201: ['workin']
Sentiment parts for row 24202: ['till', 'th']
Sentiment parts for row 24203: ['love', 'wait']
Sentiment parts for row 24204: ['gorgeous', 'two', 'g']
Sentiment parts for row 24205: ['g', 'lol', 'mornin']
Sentiment parts for row 24206: ['mi']
Sentiment parts for row 24207: ['good', 'g', 'really good', 'recommend']
Sentiment parts for row 24208: ['omg']
Sentiment parts for row 24209: ['3']
Sentiment parts for row 24210: []
Sentiment parts for row 24211: ['dear', 'de']
Sentiment parts for row 24212: ['hope', 'better', 'feel better']
Sentiment parts for row 24213: ['tho', 'ed', 'released', 'go', 'fo', 'th']
Sentiment parts for row 24214: ['sa', 'hey', 'ah', 'fo', 'th']
Sentiment parts for row 24215: ['great', 'g', 'ever', 'night']
Sentiment parts for row 24216: ['th', 'par']
Sentiment parts for row 24217: ['great', 'g', 'day']
Sentiment parts for row 24218: ['ants', 'go']
Sentiment parts for row 24219: ['g', 'ooh', 'mi']
Sentiment parts for row 242

Sentiment parts for row 25198: []
Sentiment parts for row 25199: []
Sentiment parts for row 25200: ['hide', 'chan', 'http', 'th', 'de']
Sentiment parts for row 25201: ['g']
Sentiment parts for row 25202: ['die', 'kill']
Sentiment parts for row 25203: ['love', 'loved', 'loves', 'perfect', 'g', 'mom', 'owe', 'rs', 'new']
Sentiment parts for row 25204: ['3', 'day', 'rs']
Sentiment parts for row 25205: ['http', 'th']
Sentiment parts for row 25206: []
Sentiment parts for row 25207: ['ah', 'yeah']
Sentiment parts for row 25208: ['learning', 'fo']
Sentiment parts for row 25209: ['g', 'night', 'rs']
Sentiment parts for row 25210: ['better', 'would', 'better believe', 'mi']
Sentiment parts for row 25211: ['g', 'accomplished']
Sentiment parts for row 25212: ['ed', 'de']
Sentiment parts for row 25213: ['fo', 'th', 'de']
Sentiment parts for row 25214: ['live', 'new', 'like']
Sentiment parts for row 25215: ['goin', 'going', 'go']
Sentiment parts for row 25216: ['thanks', 'thank', 'mi']
Sentiment pa

Sentiment parts for row 26070: ['ed', 'follow', 'hey', 'fo', 'th', 'de']
Sentiment parts for row 26071: []
Sentiment parts for row 26072: ['c']
Sentiment parts for row 26073: ['ed', 'happened', 'happen', 'sp', 'th']
Sentiment parts for row 26074: ['mi', 'like']
Sentiment parts for row 26075: ['sad', 'c']
Sentiment parts for row 26076: ['im', 'stop', 'c']
Sentiment parts for row 26077: ['nt']
Sentiment parts for row 26078: ['hey']
Sentiment parts for row 26079: ['un', 'th', 'watch']
Sentiment parts for row 26080: ['dead', 'officially', 'fish', 'c']
Sentiment parts for row 26081: ['bored']
Sentiment parts for row 26082: ['th']
Sentiment parts for row 26083: ['upset', '24']
Sentiment parts for row 26084: ['http']
Sentiment parts for row 26085: ['loving', 'lovin', 'g']
Sentiment parts for row 26086: ['sad']
Sentiment parts for row 26087: ['nervous', 'im', 'nt']
Sentiment parts for row 26088: ['poor', 'dog', 'rly', 'c']
Sentiment parts for row 26089: ['tho', 'indoors', 'th', 'pr']
Sentiment

Sentiment parts for row 27024: ['awesome', 'finally', 'g']
Sentiment parts for row 27025: ['happy', 'happy mother', 'mothers', 'happy mothers', 'g', 'happy mothers day', 'day', 'rs', 'mother']
Sentiment parts for row 27026: ['slow', 'losing', 'im']
Sentiment parts for row 27027: []
Sentiment parts for row 27028: ['nachos', 'th', 'de']
Sentiment parts for row 27029: ['wait', '3', 'rs']
Sentiment parts for row 27030: ['g', 'day']
Sentiment parts for row 27031: ['sick', 'im', 'nt', 'itchy', 'half', 'rly', 'c']
Sentiment parts for row 27032: ['g', 'aww']
Sentiment parts for row 27033: ['worry', 'day']
Sentiment parts for row 27034: ['training', 'raining', 'http', 'mile', 'fo', 'un', 'th']
Sentiment parts for row 27035: ['sa', 'watching', 'watchin', 'watch']
Sentiment parts for row 27036: ['play', 'http', 'fo', 'un']
Sentiment parts for row 27037: ['watching', 'watchin', 'watch']
Sentiment parts for row 27038: ['welcome', 'wait']
Sentiment parts for row 27039: ['bad', 'feel', 'feeling', 'wa

In [14]:
#Applying preprocessing function on the text column
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

# Converting sentiments to numerical form
sentiment_mapping = {'neutral': 0, 'positive': 1, 'negative': -1}
test_df['sentiment'] = test_df['sentiment'].map(sentiment_mapping)
print("Number of rows before dropna:", train_df.shape[0])

test_df.dropna(inplace=True)
print("Number of rows after dropna:", train_df.shape[0])

# Print the preprocessed test dataset
print(test_df.head())

Number of rows before dropna: 27480
Number of rows after dropna: 27480
       textID                                               text  sentiment  \
0  f87dea47db  Last session of the day  http://twitpic.com/67ezh          0   
1  96d74cb729   Shanghai is also really exciting (precisely -...          1   
2  eee518ae67  Recession hit Veronique Branquinho, she has to...         -1   
3  01082688c6                                        happy bday!          1   
4  33987a8ee5             http://twitpic.com/4w75p - I like it!!          1   

                                        cleaned_text  
0            last session day http twitpic com 67ezh  
1  shanghai also really exciting precisely skyscr...  
2  recession hit veronique branquinho quit compan...  
3                                         happy bday  
4                        http twitpic com 4w75p like  


In [15]:
# Transform the preprocessed text data into feature vectors
X_test = vectorizer.transform(test_df['cleaned_text']) 

# Use the trained model to predict the sentiment of the test dataset
y_pred = log_reg.predict(X_test)  # Apply the trained model to the test dataset


# Print the predicted sentiment labels for the test dataset
print(y_pred)

[ 0  1 -1 ...  0  1  0]


In [16]:
# Create an empty "selected_text" column in the test_df DataFrame
test_df['selected_text'] = ''

top_features_per_sentiment = {
    -1: feature_importance_df.loc[0].nlargest(num_top_features).index.tolist(),
    0: feature_importance_df.loc[1].nlargest(num_top_features).index.tolist(),
    1: feature_importance_df.loc[2].nlargest(num_top_features).index.tolist()
}

for index, row in test_df.iterrows():
    text = row['text']
    sentiment_label = int(row['sentiment'])  # Convert sentiment label to integer
    
    if sentiment_label in top_features_per_sentiment:
        top_features = top_features_per_sentiment[sentiment_label]  # All features for the sentiment label
        sentiment_parts = [feature for feature in top_features if feature in text]
    else:
        sentiment_parts = []  # No top features available for the sentiment label
    
    # Assign the sentiment parts to the "selected_text" column
    test_df.loc[index, 'selected_text'] = ' '.join(sentiment_parts)

# Create the submission DataFrame with the desired columns
submission_df = test_df[['textID', 'text', 'selected_text', 'sentiment']]

In [17]:
# Map back to original labels
sentiment_mapping = {0: 'neutral', 1: 'positive', -1: 'negative'}
submission_df.loc[:, 'sentiment'] = submission_df['sentiment'].map(sentiment_mapping)
print(submission_df)

          textID                                               text  \
0     f87dea47db  Last session of the day  http://twitpic.com/67ezh   
1     96d74cb729   Shanghai is also really exciting (precisely -...   
2     eee518ae67  Recession hit Veronique Branquinho, she has to...   
3     01082688c6                                        happy bday!   
4     33987a8ee5             http://twitpic.com/4w75p - I like it!!   
...          ...                                                ...   
3529  e5f0e6ef4b  its at 3 am, im very tired but i can`t sleep  ...   
3530  416863ce47  All alone in this old house again.  Thanks for...   
3531  6332da480c   I know what you mean. My little dog is sinkin...   
3532  df1baec676  _sutra what is your next youtube video gonna b...   
3533  469e15c5a8   http://twitpic.com/4woj2 - omgssh  ang cute n...   

         selected_text sentiment  
0              http th   neutral  
1        exciting g rs  positive  
2              shame c  negative  
3      

C:\Users\23324\AppData\Local\Temp\ipykernel_8380\2050033492.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df.loc[:, 'sentiment'] = submission_df['sentiment'].map(sentiment_mapping)


In [18]:
submission_df.to_csv('submission.csv', index=False)